In [20]:
import json

from tool.annotations_utils import read_annotations, has_intersection

title = 'Emma'
DIR_PATH = 'data/testing_sets/test_names_gold_standard_corrected'

In [23]:
def create_choices_for_label_studio(filepath):
    choices = []
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file.readlines():
            choices.append(f'<Choice value="{line.strip()}"/\n>')
    return ' '.join(choices)


print(create_choices_for_label_studio('data/lists_of_characters/' + title))

<Choice value="Emma Woodhouse"/
> <Choice value="George Knightley"/
> <Choice value="Frank Churchill"/
> <Choice value="Jane Fairfax"/
> <Choice value="Harriet Smith"/
> <Choice value="Robert Martin"/
> <Choice value="Philip Elton"/
> <Choice value="Augusta Elton"/
> <Choice value="Mrs. Weston"/
> <Choice value="Mr. Weston"/
> <Choice value="Miss Bates"/
> <Choice value="Mr. Henry Woodhouse"/
> <Choice value="Isabella Knightley"/
> <Choice value="John Knightley"/
>


In [16]:
def prepare_single_annotation(annotation_id, annotation_content):
    return {'id': annotation_id + 1,
            'data': {'text': annotation_content},
            'annotations': [
                {'model_version': 'v1', 'result': []}]}


def convert_to_label_studio_entity(entity, annotation, version=0):
    if version == 0:
        return {
                    'from_name': 'label',
                    'to_name': 'text',
                    'type': 'labels',
                    'id': str(hash(str(annotation)+str(entity))),
                    'value': {
                        'start': entity[0],
                        'end': entity[1],
                        'text': annotation['content'][entity[0]:entity[1]],
                        'labels': ['PERSON']
                    }
                }
    if version == 1:
        return {
                    'value': {
                        'start': entity[0],
                        'end': entity[1],
                        'text': annotation['content'][entity[0]:entity[1]],
                        'choices': [entity[2]]
                    },
                    'id': str(hash(str(annotation)+str(entity))),
                    'from_name': 'relevance',
                    'to_name': 'text',
                    'type': 'choices'
            }


def convert_to_label_studio_format(raw_annotations, output_path=None):
    new_annotations = []

    for anno_id, anno in enumerate(raw_annotations):
        single_annotation = prepare_single_annotation(anno_id, anno['content'])

        for entity in anno['entities']:
            new_entity_v0 = convert_to_label_studio_entity(entity, anno, 0)
            new_entity_v1 = convert_to_label_studio_entity(entity, anno, 1)

            single_annotation['annotations'][0]['result'].append(new_entity_v0)
            single_annotation['annotations'][0]['result'].append(new_entity_v1)

        new_annotations.append(single_annotation)

    if output_path:
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(json.dumps(new_annotations).encode('utf8').decode())
    return new_annotations

'/home/airi/PycharmProjects/protagonist_tagger'